In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/pruning_quantization", "")))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models, CNN_models
from utils.benchmark import Benchmark
from compact.pruning import prune_mlp_model, prune_rnn_model, prune_cnn_model
from utils.plot import plot_fpr_tpr_roc_auc

In [ ]:
conf = Config()
load_model = GRU_Models()
model_conf = load_model.gru_4
model = load_model.get_model(model_conf)
model.load()
print(conf.device)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/pruned_quantized/gru_pq.pth!
cpu


In [3]:
# data loader
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, model_type=load_model.type)

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

dataset = NetworkDataset(X_test, y_test)
loader = DataLoader(dataset, conf.batch_size, shuffle=True)

In [ ]:
# prune model
checkpoint_path = os.path.join(conf.checkpoint, "pruned_quantized", "pruned_" + model_conf["name"] + ".pth")
model.checkpoint_path = checkpoint_path

# new hidden sizes
if load_model.type == "mlp":
    pruned_model = prune_mlp_model(model.model)
elif load_model.type == "rnn":
    pruned_model = prune_rnn_model(model.model)
elif load_model.type == "cnn":
    pruned_model = prune_cnn_model(model.model)
else:
    raise ValueError("model type must be 'mlp', 'rnn' or 'cnn'!")
    

model.model = pruned_model.to(conf.device)
model.optimizer = torch.optim.AdamW(
    model.model.parameters(), 
    lr=conf.learning_rate, 
    weight_decay=conf.weight_decay
)
model.scheduler = torch.optim.lr_scheduler.ExponentialLR(
    model.optimizer, 
    gamma=conf.gamma
)
model.load()

In [ ]:
# benchmark model
name = model_conf["name"]
result_path = os.path.join(conf.benchmark_host, "pruned_quantized_model", "pruned_" + name + ".txt")
benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
benchmark()

# print and save result
benchmark.print_result()
benchmark.save()


Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/pruned_quantized/pruned_gru_pq.pth!


[W116 10:37:49.680208718 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
ERROR:2026-01-16 10:37:57 4064760:4064760 DeviceProperties.cpp:47] gpuGetDeviceCount failed with code 35


Benchmark - gru_pq model:

Memory usage (MB):
Avg memory usage: 3.221MB
Peak memory usage: 47.911MB

Model inference latency on one batch (batch size = 1):
Avg latency: 75.764ms
Min latency: 66.073ms
Max latency: 82.488ms

Model inference throughput (batch size = 1):
Throughput: 14.02 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 3.56 seconds
Average CPU usage: 35.79/96 cores

Model (gru_pq) Macro-F1, Micro-F1 and Macro ROC AUC scores:
Macro-F1 score: 0.44
Micro-F1 score: 0.94
Macro ROC AUC score: nan





/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/si